In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
from collections import Counter
import ast
import pickle
import random

warnings.filterwarnings('ignore')
tqdm.pandas()

In [7]:
# internal data setting
internal = pd.DataFrame(columns = ["customer_id", "cluster", "sequence", "names", "tfidf", "tfidf_name"])
for num in tqdm(range(33)):
    temp = pd.read_csv(f"C:/Users/amore/Changhyun/vector/result{num}.csv")
    tfidf = pd.read_csv(f"C:/Users/amore/Changhyun/named_tfidf_results{num}.csv")
    
    indice = []
    vc = temp["cluster"].value_counts()
    for i in vc.index:
        if vc.loc[i] >= 1000:
            indice.append(i)
    for i in indice:
        temp2 = temp[temp["cluster"] == i][["customer_id", "cluster", "sequence", "names"]]
        temp2["cluster"] = [str(num) + "-" + str(w) for w in temp2["cluster"]]
        temp2["tfidf"] = [list(tfidf["item" + str(i)]) for w in temp2["cluster"]]
        temp2["tfidf_name"] = [list(tfidf["name" + str(i)]) for w in temp2["cluster"]]
        internal = pd.concat([internal, temp2], axis = 0)
internal

100%|██████████| 33/33 [00:13<00:00,  2.49it/s]


,customer_id,cluster,sequence,names,tfidf,tfidf_name
0,bcca79b2d92905ae84afff9fc7796efc3e416294ecb1f9...,0-0,"['111772532', '131173347', '111772532']","['이니스프리 스키니꼼꼼카라', '이니스프리 심플라벨워터프루프펜슬라이너', '이니스...","[131171316, 131170000, 111706495, 111772532, 1...","[131171316, 이니스프리 스키니꼼꼼카라ZERO, 이니스프리 에코뷰티툴미니파우..."
1,9c389c15d72731f80dc6bcb47d480372f46d05c3be5a1c...,0-0,"['111501723', '111501723', '111501723', '11150...","['이니스프리 노세범미네랄파우더', '이니스프리 노세범미네랄파우더', '이니스프리 ...","[131171316, 131170000, 111706495, 111772532, 1...","[131171316, 이니스프리 스키니꼼꼼카라ZERO, 이니스프리 에코뷰티툴미니파우..."
2,969fcec0dac45369dbdbe3d26507ffaeff3abb1b697aed...,0-0,"['111501723', '111501723', '111501723']","['이니스프리 노세범미네랄파우더', '이니스프리 노세범미네랄파우더', '이니스프리 ...","[131171316, 131170000, 111706495, 111772532, 1...","[131171316, 이니스프리 스키니꼼꼼카라ZERO, 이니스프리 에코뷰티툴미니파우..."
5,085c90e260b9c878410f8f2a9ac281b833fe6a6b5abbe3...,0-0,"['111772219', '111772219', '111772219']","['이니스프리 에코납작브로우펜슬', '이니스프리 에코납작브로우펜슬', '이니스프리 ...","[131171316, 131170000, 111706495, 111772532, 1...","[131171316, 이니스프리 스키니꼼꼼카라ZERO, 이니스프리 에코뷰티툴미니파우..."
6,f104166ff22315c3f594b4eb829e0f6c67d51faead20e7...,0-0,"['111774548', '111772219', '111501723', '11177...","['이니스프리 마이아이섀도우(쉬머)', '이니스프리 에코납작브로우펜슬', '이니스프...","[131171316, 131170000, 111706495, 111772532, 1...","[131171316, 이니스프리 스키니꼼꼼카라ZERO, 이니스프리 에코뷰티툴미니파우..."
...,...,...,...,...,...,...
29853,abf4f7dac660f9c8ccd8984632fe927a5b54ece997adb0...,32-1,"['131173534', '111770997', '111770120']","['이니스프리 듀이틴트립밤', '이니스프리 잇츠리얼스퀴즈마스크', '이니스프리 그린...","[111770997, 111506404, 111771128, 111506462, 1...","[이니스프리 잇츠리얼스퀴즈마스크, 이니스프리 에코뷰티툴일반화장솜, 이니스프리 스킨클..."
29856,a786eb962a3c3c8d3324d9d98430e80e277d8aac6c2f65...,32-1,"['111771038', '111770997', '111772219']","['이니스프리 꽃송이버섯바이탈로션', '이니스프리 잇츠리얼스퀴즈마스크', '이니스프...","[111770997, 111506404, 111771128, 111506462, 1...","[이니스프리 잇츠리얼스퀴즈마스크, 이니스프리 에코뷰티툴일반화장솜, 이니스프리 스킨클..."
29859,958a81a9b8db88009164020a17738169346fccd86b5a3b...,32-1,['111770997'],['이니스프리 잇츠리얼스퀴즈마스크'],"[111770997, 111506404, 111771128, 111506462, 1...","[이니스프리 잇츠리얼스퀴즈마스크, 이니스프리 에코뷰티툴일반화장솜, 이니스프리 스킨클..."
29863,f985e1ef7182c15d9e55f6dbdd1edaff598984a510fae3...,32-1,"['111774747', '111774226', '111774339', '11177...","['이니스프리 블루베리5.5클렌저', '이니스프리 애플씨드립앤아이리무버', '111...","[111770997, 111506404, 111771128, 111506462, 1...","[이니스프리 잇츠리얼스퀴즈마스크, 이니스프리 에코뷰티툴일반화장솜, 이니스프리 스킨클..."


In [21]:
df = pd.read_csv("total_samples2.csv")
lit = list(df["customer_id"].unique())
ic = df["internal_cluster"].value_counts()

internal["used"] = [1 if i in lit else 0 for i in tqdm(internal["customer_id"])]
internal[internal["used"] == 1].reset_index(drop = True)
del internal["used"]
internal

100%|██████████| 355448/355448 [05:23<00:00, 1099.59it/s]


,customer_id,cluster,sequence,names,tfidf,tfidf_name
0,bcca79b2d92905ae84afff9fc7796efc3e416294ecb1f9...,0-0,"['111772532', '131173347', '111772532']","['이니스프리 스키니꼼꼼카라', '이니스프리 심플라벨워터프루프펜슬라이너', '이니스...","[131171316, 131170000, 111706495, 111772532, 1...","[131171316, 이니스프리 스키니꼼꼼카라ZERO, 이니스프리 에코뷰티툴미니파우..."
1,9c389c15d72731f80dc6bcb47d480372f46d05c3be5a1c...,0-0,"['111501723', '111501723', '111501723', '11150...","['이니스프리 노세범미네랄파우더', '이니스프리 노세범미네랄파우더', '이니스프리 ...","[131171316, 131170000, 111706495, 111772532, 1...","[131171316, 이니스프리 스키니꼼꼼카라ZERO, 이니스프리 에코뷰티툴미니파우..."
2,969fcec0dac45369dbdbe3d26507ffaeff3abb1b697aed...,0-0,"['111501723', '111501723', '111501723']","['이니스프리 노세범미네랄파우더', '이니스프리 노세범미네랄파우더', '이니스프리 ...","[131171316, 131170000, 111706495, 111772532, 1...","[131171316, 이니스프리 스키니꼼꼼카라ZERO, 이니스프리 에코뷰티툴미니파우..."
5,085c90e260b9c878410f8f2a9ac281b833fe6a6b5abbe3...,0-0,"['111772219', '111772219', '111772219']","['이니스프리 에코납작브로우펜슬', '이니스프리 에코납작브로우펜슬', '이니스프리 ...","[131171316, 131170000, 111706495, 111772532, 1...","[131171316, 이니스프리 스키니꼼꼼카라ZERO, 이니스프리 에코뷰티툴미니파우..."
6,f104166ff22315c3f594b4eb829e0f6c67d51faead20e7...,0-0,"['111774548', '111772219', '111501723', '11177...","['이니스프리 마이아이섀도우(쉬머)', '이니스프리 에코납작브로우펜슬', '이니스프...","[131171316, 131170000, 111706495, 111772532, 1...","[131171316, 이니스프리 스키니꼼꼼카라ZERO, 이니스프리 에코뷰티툴미니파우..."
...,...,...,...,...,...,...
29853,abf4f7dac660f9c8ccd8984632fe927a5b54ece997adb0...,32-1,"['131173534', '111770997', '111770120']","['이니스프리 듀이틴트립밤', '이니스프리 잇츠리얼스퀴즈마스크', '이니스프리 그린...","[111770997, 111506404, 111771128, 111506462, 1...","[이니스프리 잇츠리얼스퀴즈마스크, 이니스프리 에코뷰티툴일반화장솜, 이니스프리 스킨클..."
29856,a786eb962a3c3c8d3324d9d98430e80e277d8aac6c2f65...,32-1,"['111771038', '111770997', '111772219']","['이니스프리 꽃송이버섯바이탈로션', '이니스프리 잇츠리얼스퀴즈마스크', '이니스프...","[111770997, 111506404, 111771128, 111506462, 1...","[이니스프리 잇츠리얼스퀴즈마스크, 이니스프리 에코뷰티툴일반화장솜, 이니스프리 스킨클..."
29859,958a81a9b8db88009164020a17738169346fccd86b5a3b...,32-1,['111770997'],['이니스프리 잇츠리얼스퀴즈마스크'],"[111770997, 111506404, 111771128, 111506462, 1...","[이니스프리 잇츠리얼스퀴즈마스크, 이니스프리 에코뷰티툴일반화장솜, 이니스프리 스킨클..."
29863,f985e1ef7182c15d9e55f6dbdd1edaff598984a510fae3...,32-1,"['111774747', '111774226', '111774339', '11177...","['이니스프리 블루베리5.5클렌저', '이니스프리 애플씨드립앤아이리무버', '111...","[111770997, 111506404, 111771128, 111506462, 1...","[이니스프리 잇츠리얼스퀴즈마스크, 이니스프리 에코뷰티툴일반화장솜, 이니스프리 스킨클..."


In [23]:
res = pd.DataFrame(columns = internal.columns)
for i in ic.index:
    num = int(ic.loc[i] * 0.72)
    temp = internal[internal["cluster"] == i]
    res = pd.concat([res, temp.iloc[:num]], axis = 0)
res = res.reset_index(drop = True)
res

,customer_id,cluster,sequence,names,tfidf,tfidf_name
0,4a5500c5c3e4ded7581fcdd9df52ddeb56b09e02dac345...,6-0,"['131170811', '111703767', '131174337', '13117...","['이니스프리 포레스트포맨올인원에센스', '이니스프리 스페셜케어마스크', '이니스프...","[131172880, 131174283, 131172879, 131171020, 1...","[131172880, 이니스프리 시그니처핸드크림, 131172879, 이니스프리 제..."
1,86e5334af3f0cdeb0c7bfcb8ff39ae27fe00d6591031fe...,6-0,"['131172563', '111705279', '131174322', '11177...","['이니스프리 블랙티유스인핸싱앰플', '이니스프리 더그린티씨드세럼', '이니스프리 ...","[131172880, 131174283, 131172879, 131171020, 1...","[131172880, 이니스프리 시그니처핸드크림, 131172879, 이니스프리 제..."
2,c0943a98be324033db815750d6a086cb9e7f55de133083...,6-0,"['131174827', '111705459', '131174661', '13117...","['이니스프리 콜라겐그린티세라마이드탄력장벽크림', '이니스프리 올리브리얼클렌징폼',...","[131172880, 131174283, 131172879, 131171020, 1...","[131172880, 이니스프리 시그니처핸드크림, 131172879, 이니스프리 제..."
3,72ffa2bac77853875c7921a18d6636462a32fa30e6b1cf...,6-0,"['111770127', '111770183', '111770156', '11170...","['이니스프리 비자트러블훼이셜폼', '이니스프리 유채꿀립밤', '이니스프리 포레스트...","[131172880, 131174283, 131172879, 131171020, 1...","[131172880, 이니스프리 시그니처핸드크림, 131172879, 이니스프리 제..."
4,1796b998661dd9b51337c77465b2ca61683553bfd41df0...,6-0,"['131173534', '131174827', '111770997', '13117...","['이니스프리 듀이틴트립밤', '이니스프리 콜라겐그린티세라마이드탄력장벽크림', '이...","[131172880, 131174283, 131172879, 131171020, 1...","[131172880, 이니스프리 시그니처핸드크림, 131172879, 이니스프리 제..."
...,...,...,...,...,...,...
16616,e72f252c42f232845cf0afca5b55b5210c1c8146ab1235...,4-0,"['111501723', '111501723', '111501723', '11150...","['이니스프리 노세범미네랄파우더', '이니스프리 노세범미네랄파우더', '이니스프리 ...","[131173960, 111771793, 131173534, 111770183, 1...","[이니스프리 비타엔자임토닝앰플, 이니스프리 리얼컬러네일, 이니스프리 듀이틴트립밤, ..."
16617,51c48e8092867f4c7c9f2280546ae0169cbb40179210c8...,4-0,['111501723'],['이니스프리 노세범미네랄파우더'],"[131173960, 111771793, 131173534, 111770183, 1...","[이니스프리 비타엔자임토닝앰플, 이니스프리 리얼컬러네일, 이니스프리 듀이틴트립밤, ..."
16618,da2f3601c21387382dbb61ff761e0666e987b42da4d58e...,4-0,"['131172573', '131174670', '131173960', '11170...","['이니스프리 프루티스퀴즈틴트', '이니스프리 라이트 피팅 파운데이션', '이니스프...","[131173960, 111771793, 131173534, 111770183, 1...","[이니스프리 비타엔자임토닝앰플, 이니스프리 리얼컬러네일, 이니스프리 듀이틴트립밤, ..."
16619,c3f8d7feb3e235ea9d7c4ed81583a3608ee0f8e3797c7a...,4-0,['111501723'],['이니스프리 노세범미네랄파우더'],"[131173960, 111771793, 131173534, 111770183, 1...","[이니스프리 비타엔자임토닝앰플, 이니스프리 리얼컬러네일, 이니스프리 듀이틴트립밤, ..."


In [31]:
res = res[["customer_id", "cluster", "names"]]
res.columns = ["customer_id", "internal_cluster", "internal_cluster_info"]
res["external_cluster"] = np.nan
res["external_cluster_info"] = np.nan

df = pd.concat([df, res], axis = 0).reset_index(drop = True)
df

,customer_id,internal_cluster,internal_cluster_info,external_cluster,external_cluster_info
0,699cb2f62f368a4bcb8faf4cec63d6592f6ef5e48cbd60...,6-0,"['IF.리스테이 카밍 컨디셔너 480ml(R)', 'IF. 아일넘버 핸드크림 씨즈...",0.0,"('PER_OTT 상', 'PER_MDLV 상', 'PER_PRC_SNSV 상')"
1,d5ecc2bf8426946958f568e86f6f17adbec6540a308959...,6-0,"['IF.리스테이 카밍 컨디셔너 480ml(R)', 'IF. 아일넘버 핸드크림 씨즈...",0.0,"('PER_OTT 상', 'PER_MDLV 상', 'PER_PRC_SNSV 상')"
2,662f112160158cbb9da36887f24a2581696ac78085a82f...,6-0,"['IF.리스테이 카밍 컨디셔너 480ml(R)', 'IF. 아일넘버 핸드크림 씨즈...",0.0,"('PER_OTT 상', 'PER_MDLV 상', 'PER_PRC_SNSV 상')"
3,2a9a580f05e866ae2f4127342ee6905590460e0c7048db...,6-0,"['IF.리스테이 카밍 컨디셔너 480ml(R)', 'IF. 아일넘버 핸드크림 씨즈...",0.0,"('PER_OTT 상', 'PER_MDLV 상', 'PER_PRC_SNSV 상')"
4,e314e8d74ebbd0458995b3536320d9794354831db4c0b0...,6-0,"['IF.리스테이 카밍 컨디셔너 480ml(R)', 'IF. 아일넘버 핸드크림 씨즈...",0.0,"('PER_OTT 상', 'PER_MDLV 상', 'PER_PRC_SNSV 상')"
...,...,...,...,...,...
39722,e72f252c42f232845cf0afca5b55b5210c1c8146ab1235...,4-0,"['이니스프리 노세범미네랄파우더', '이니스프리 노세범미네랄파우더', '이니스프리 ...",NaN,NaN
39723,51c48e8092867f4c7c9f2280546ae0169cbb40179210c8...,4-0,['이니스프리 노세범미네랄파우더'],NaN,NaN
39724,da2f3601c21387382dbb61ff761e0666e987b42da4d58e...,4-0,"['이니스프리 프루티스퀴즈틴트', '이니스프리 라이트 피팅 파운데이션', '이니스프...",NaN,NaN
39725,c3f8d7feb3e235ea9d7c4ed81583a3608ee0f8e3797c7a...,4-0,['이니스프리 노세범미네랄파우더'],NaN,NaN


In [33]:
df.to_csv("final_sample_20241211.csv", encoding = "utf-8-sig")